 # 📌 Notebook 2: Retrieve and Test Recommendations (GTE-small)
This notebook allows you to enter a shot scenario and retrieve semantically similar historical shots using Qdrant with GTE-small embeddings.
   

## Initial Setup

In [9]:
# Select embedding model
# Options: 'e5-base-v2', 'bge-base-en-v1.5'
embedding_model_choice = 'bge-base-en-v1.5'
descriptive_collection_name = f"golf_shot_vectors_{embedding_model_choice.replace('-', '_')}"

from sentence_transformers import SentenceTransformer

def load_embedding_model(name: str):
    if name == 'e5-base-v2':
        return SentenceTransformer('intfloat/e5-base-v2')
    elif name == 'bge-base-en-v1.5':
        return SentenceTransformer('BAAI/bge-base-en-v1.5')
    else:
        raise ValueError(f'Unsupported model: {name}')

model = load_embedding_model(embedding_model_choice)

In [13]:
# Embed the query using the selected embedding model

def get_embedding(text):
    if embedding_model_choice.startswith("e5"):
        return model.encode(f"query: {text}")
    else:
        return model.encode(text)

# Define a test query
# query = "What club should I use from 145 yards if I want to avoid a slice?"
# query = "What club should I use from 145 yards?"
# query = "Find shots from 140 to 150 yards where the shape was a slice. Return the club used and distance."

# preprocessed-style query:
query = "The golfer is planning a 145-yard shot and wants to avoid a slice."

query_vector = get_embedding(query)


In [11]:
# Collect the Qdrant API key
from getpass import getpass

qdrant_api_key = getpass('🔑 Enter your Qdrant API Key: ')

In [14]:
# Retrieve top 5 most similar entries
from qdrant_client import QdrantClient

client = QdrantClient(
    url="https://6f592f43-f667-4234-ad3a-4f15ed5882ef.us-west-2-0.aws.cloud.qdrant.io:6333",
    api_key=qdrant_api_key
)

# collection_name = descriptive_collection_name

search_result = client.query_points(
    collection_name=descriptive_collection_name,
    query=query_vector,  # Pass the vector directly
    limit=20,
    with_payload=True
    )
    
# Access the points from the QueryResponse
for point in search_result.points:
    print(f"Score: {point.score:.4f} | Payload: {point.payload}")


Score: 0.7182 | Payload: {'text': 'On 2024-09-26 15:21:15, the golfer hit a shot 145.06 yards with a carry of 127.38 yards using a 9 Iron (** Top Golf - Short Irons - 9I). The shot was classified as Slice. The known contributing factors to this result were: Ball speed: 85.43 mph. Club speed: 74.94 mph. Spin rate: 2222 rpm. Attack angle: -2.65 degrees. Descent angle: 40.41 degrees.'}
Score: 0.7137 | Payload: {'text': 'On 2024-09-05 13:39:59, the golfer hit a shot 114.56 yards with a carry of 99.57 yards using a Approach Wedge (48 Degree). The shot was classified as Slice. The known contributing factors to this result were: Ball speed: 74.27 mph. Club speed: 77.26 mph. Spin rate: 1154 rpm. Attack angle: 4.26 degrees. Descent angle: 41.06 degrees.'}
Score: 0.7117 | Payload: {'text': 'On 2024-09-05 13:30:59, the golfer hit a shot 94.94 yards with a carry of 83.81 yards using a Sand Wedge (56 Degree). The shot was classified as Slice. The known contributing factors to this result were: Ball